#BaseLine

In [58]:
import numpy as np 
import pandas as pd 
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import SGDClassifier
from sklearn.naive_bayes import MultinomialNB, BernoulliNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVC
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
import random
from sklearn.model_selection import cross_validate, cross_val_predict, cross_val_score
from sklearn import metrics

In [59]:
!gdown --id 1ewF8bx1KBLZTTeKWg4v5hzpo-cfHdy1O -O /content/CY_data.txt

Downloading...
From: https://drive.google.com/uc?id=1ewF8bx1KBLZTTeKWg4v5hzpo-cfHdy1O
To: /content/CY_data.txt
100% 345k/345k [00:00<00:00, 61.5MB/s]


In [60]:
df_cy = pd.read_csv("/content/CY_data.txt")

In [61]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(df_cy, test_size=0.2, stratify=df_cy['class'])

This Function will run the passed model and do 10 folds cross val for the following models:

1.LinearSVC

2.MultinomialNB

3.BernoulliNB

4.SGDClassifier 

5.DecisionTreeClassifier

6.RandomForestClassifier

7.KNeighborsClassifier

We will use tf-idf to process our data with analyzer : "word", and n gram "1"

In [62]:
def run_models(my_classifier, name, analyzer = "word", ngram = 1):
    x_train = train['tweet']
    y_train = train['class']
    x_test = test['tweet']
    y_test = test['class']
    print('classifier:', my_classifier.__class__.__name__)
    print('------------------------------------')




    vectorizer = TfidfVectorizer(min_df=0.00001, max_df=0.99,
                                 analyzer=analyzer, lowercase=False,
                                 ngram_range=(1, ngram))

    X = vectorizer.fit_transform(x_train)
    xtest = vectorizer.transform(x_test)

    my_classifier.fit(X,y_train)
    
    scores = cross_validate(my_classifier, X, y_train, cv=10,
                      scoring=('accuracy', 'f1_weighted')
                      )
    
    y_train_pred = my_classifier.predict(X)
    print(f"The results accuracy on the training data {accuracy_score(y_train_pred, y_train)}")
    print(f"The Cross validation Accuracy is {scores['test_accuracy'].mean()}, The F1-score is {scores['test_f1_weighted'].mean()}")
    print("The Results on test data")
    y_predicted = my_classifier.predict(xtest)
    print("accuracy: ")
    print(accuracy_score(y_test, y_predicted))
    print("f1_score: ")
    print(f1_score(y_test, y_predicted, average="weighted"))


In [63]:
classifiers = [LinearSVC(), MultinomialNB(),
               BernoulliNB(), SGDClassifier(), DecisionTreeClassifier(max_depth=4),
               RandomForestClassifier(max_depth=5, n_estimators=10, max_features=1),
               KNeighborsClassifier(3)
               ]

for model in classifiers:
    alg_name = model.__class__.__name__
    run_models(model, alg_name, analyzer= "word", ngram=1)

classifier: LinearSVC
------------------------------------
The results accuracy on the training data 1.0
The Cross validation Accuracy is 0.5673993358239934, The F1-score is 0.5545092145774995
The Results on test data
accuracy: 
0.5545454545454546
f1_score: 
0.5522760691964762
classifier: MultinomialNB
------------------------------------
The results accuracy on the training data 0.9371584699453552
The Cross validation Accuracy is 0.5637463677874637, The F1-score is 0.508884438528554
The Results on test data
accuracy: 
0.5927272727272728
f1_score: 
0.5485148311605621
classifier: BernoulliNB
------------------------------------
The results accuracy on the training data 0.8292349726775956
The Cross validation Accuracy is 0.52730801162308, The F1-score is 0.4617566768565672
The Results on test data
accuracy: 
0.5236363636363637
f1_score: 
0.44947261789122256
classifier: SGDClassifier
------------------------------------
The results accuracy on the training data 1.0
The Cross validation Ac

#Here we got our baseline result and we will move try to improve, best model was LinearSVC and we will keep using it.

##accuracy = 0.5872
##f1-score = 0.5325



seems like we got a poor accuracy on test and dev set and very high accuracy on training data --> which means we have overfit, and its more likely beceause of we are out-of-words

we can solve than buy using char n-gram instead of words

In [64]:
#Here we changed our analyzer to "char" and increased the ngram 
print("model : LinearSVC")

run_models(LinearSVC(), alg_name, analyzer= "char", ngram=6)

model : LinearSVC
classifier: LinearSVC
------------------------------------
The results accuracy on the training data 0.9990892531876139
The Cross validation Accuracy is 0.6170298879202989, The F1-score is 0.6033680124246437
The Results on test data
accuracy: 
0.6418181818181818
f1_score: 
0.6389297975534348


#After trail and error we reached the 
 
#accuracy 0.6272  
#f1 score of 0.61675

lets Try do propressing to our data

In [65]:
!pip install pyarabic emoji 
!pip install pystemmer
!pip install optuna==2.3.0
!pip install transformers==4.2.1

In [66]:
import numpy as np
import pandas as pd
import pyarabic.araby as ar
import re , emoji, Stemmer, functools, operator, string
import torch , optuna, gc, random, os
from sklearn.metrics import classification_report, accuracy_score, f1_score, confusion_matrix, precision_score , recall_score
from sklearn.utils import resample
from nltk.corpus import stopwords
from nltk import word_tokenize
import nltk
import re


nltk.download("stopwords")
nltk.download("punkt")


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [67]:

stopwords = stopwords.words("arabic")

def remove_stopwords(text):
  text_tokens = word_tokenize(text)

  tokens_without_sw = [word for word in text_tokens if not word in stopwords]

  return " ".join(tokens_without_sw)

def no_english(text):
	text = re.sub("[a-zA-Z]+", "",text)
	return text

In [68]:
st =  Stemmer.Stemmer('arabic')
def data_cleaning (text):
  text = re.sub(r'^https?:\/\/.*[\r\n]*', '', text, flags=re.MULTILINE) #Removing links Https
  text = re.sub(r'^http?:\/\/.*[\r\n]*', '', text, flags=re.MULTILINE) #removing Http links
  text = re.sub(r"http\S+", "", text) #removing links
  text = re.sub(r"https\S+", "", text) #removing links
  text = re.sub(r'\s+', ' ', text) # removing more than space to on space
  text = re.sub("(\s\d+)","",text)  #removing digits
  text = re.sub(r"$\d+\W+|\b\d+\b|\W+\d+$", "", text)
  text = re.sub("\d+", " ", text) #removing digits
  text = ar.strip_tashkeel(text) #removing Tshkeel
  text = ar.strip_tatweel(text) 
  text = text.replace("#", " ");  #removing #
  text = text.replace("@", " "); #removing @
  text = text.replace("_", " "); #removing _
  translator = str.maketrans('', '', string.punctuation)
  text = text.translate(translator)
  em = text 
  em_split_emoji = emoji.get_emoji_regexp().split(em)
  em_split_whitespace = [substr.split() for substr in em_split_emoji]
  em_split = functools.reduce(operator.concat, em_split_whitespace) #processing emojis
  text = " ".join(em_split)
  text = re.sub(r'(.)\1+', r'\1', text)
  text_stem = " ".join([st.stemWord(i) for i in text.split()])
  text = text +" "+ text_stem
  text = text.replace("آ", "ا")
  text = text.replace("إ", "ا")
  text = text.replace("أ", "ا")
  text = text.replace("ؤ", "و")
  text = text.replace("ئ", "ي")
  text = no_english(text) #remocing remaining english words
  text = remove_stopwords(text) #removing stopwords
  return text

In [69]:
# Cleaning Training Data 
train['tweet'] = train['tweet'].apply(lambda x:   data_cleaning(x))

train.columns = ['tweet','class']

train['tweet'].head(20)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:19: DeprecationWarning: 'emoji.get_emoji_regexp()' is deprecated and will be removed in version 2.0.0. If you want to remove emoji from a string, consider the method emoji.replace_emoji(str, replace='').
To hide this warning, pin/downgrade the package to 'emoji~=1.6.3'


1792    فانظروا تسلكون بالتدقيق كجهلاء كحكماء فانظر تس...
1785         ويسعدكم اجمعين امين يارب يسعد اجمع امين يارب
759     بقيت بحس ان الايموشن دا قليل الادب او بيدي ايح...
2324    دا ميمنعش انى بعشق ال فى اسكندرية دا ميمنعش ان...
51       دا الدين عرف الحر مذلة مر دا دين عرف الحر مذل مر
137     سنه وانتى طيبه يامديره واحه الاسلام وربنا ىزيد...
1506    اه باين واله توضيح هعمل بنصيحتك حاضر شوفت بسمع...
1451    بارك اله فيكم اخي شمالي … ربنا يستجيب دعاءكم ي...
4                                     انا هنتحر انا هنتحر
888     الهم اغفر ولجميع امة المسلمين واشفي مرضانا ومر...
2440       مادة بتنفهم بندرس ليش هيه ماد تنف ندرس ليش هيه
49      جيل الايباد بمناسبة الامتحانات مامته بتقولوا ذ...
698     اما نظم العدالة الاجتماعية فتبعث الاطمينان وال...
1243    قيادات الاخوان يعلمون انهم مدانون الجرايم يحاك...
35      تياس اذا حرمك اله تحب تحزن اذا انجبرت التعايش ...
1808    طلب حسام حسن اداره الاتحاد السكندري التعاقد ال...
837     اذا بتنحاز تماما لهم، نحاز فيهن فلسطين وحر بلد...
551     انجزوا

In [70]:
#Cleaning the test data
test['tweet'] = test['tweet'].apply(lambda x:   data_cleaning(x))

test.columns = ['tweet','class']

test['tweet'].head(20)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:19: DeprecationWarning: 'emoji.get_emoji_regexp()' is deprecated and will be removed in version 2.0.0. If you want to remove emoji from a string, consider the method emoji.replace_emoji(str, replace='').
To hide this warning, pin/downgrade the package to 'emoji~=1.6.3'


2087                         صبح علي جيرانك مثﻻ صبح جيران
818     ان حد ميت يوحشك تفضل تفتكر مواقف ليه معاك تضحك...
226                                   يخليه فحالة يخل حال
352                         كسم مدريد ياعم كسم مدريد ياعم
367             الصبر لسه ساكت ويلي سكاته لسه ساك ويل سكا
907     الهم اغنا بحلالك حرامك وبفضلك عمن سواك الهم اغ...
2688    جزء الامبراطورية البريطانية مكتوب بشكل رييسي ب...
175            الثانوية دى بتنمى الغباء ثانو دي تنمي غباء
2735    عقباليك تكبد قابينة ونفرحو بيكم عقبال تكبد علي...
1414    اتخصي جراحه عشان اشغلكفي المستشفي بتاعتي اتخص ...
143     وان يرزقك خدمتها علي اكمل وجه ويكتب اله الثواب...
2680    نريد ان نعرف او يقولو موازنة مجلس النواب البنا...
2626    بفرح حد بحبه يفرح اكتر فرحته شخصيا فرح حد بحب ...
19      السكينة لباسه والبر شعاره والتقوى ضميره والحكم...
2089    معالي النقيب العام سامح عاشور نقيب المحامين ور...
383     ريتك معايا تطوفي وسط البشر والزحام وتشوفي حيرت...
1251    بحبكم كلكم قرايبي، ابشروا قدها مدام جيجي بحب ك...
2471    الاهتم

In [71]:
#Lets run the model
run_models(LinearSVC(), alg_name, analyzer= "char", ngram=6)

classifier: LinearSVC
------------------------------------
The results accuracy on the training data 0.9986338797814208
The Cross validation Accuracy is 0.6029202988792031, The F1-score is 0.5900329808462488
The Results on test data
accuracy: 
0.6381818181818182
f1_score: 
0.6288644509612252


we got no improvments, lets try adding more data

## ASTD dataset

In [72]:
!git clone https://github.com/mahmoudnabil/ASTD.git

fatal: destination path 'ASTD' already exists and is not an empty directory.


##ArSaS dataset

In [73]:
!gdown --id 1MwPiwrWp2gHWpsrUTAUcheTQiniFbBv9 -O /content/ASTD/data/ArSAS.txt

Downloading...
From: https://drive.google.com/uc?id=1MwPiwrWp2gHWpsrUTAUcheTQiniFbBv9
To: /content/ASTD/data/ArSAS.txt
100% 5.78M/5.78M [00:00<00:00, 53.6MB/s]


In [74]:
df_ArSAS = pd.read_csv("/content/ASTD/data/ArSAS.txt", sep="\t")

In [75]:
df_ArSAS = df_ArSAS[['Tweet_text', 'Sentiment_label']]

In [76]:
df_ArSAS.Sentiment_label.value_counts()

Negative    7384
Neutral     6894
Positive    4400
Mixed       1219
Name: Sentiment_label, dtype: int64

In [77]:
#dropping mixed label
df_ArSAS.drop(df_ArSAS[df_ArSAS.Sentiment_label == 'Mixed'].index, inplace=True)

In [78]:
#checking the dataframe labels
df_ArSAS.Sentiment_label.value_counts()

Negative    7384
Neutral     6894
Positive    4400
Name: Sentiment_label, dtype: int64

In [79]:
df_tweets = pd.read_csv("/content/ASTD/data/Tweets.txt", sep="\t", on_bad_lines='skip', header=None)

In [80]:
df_tweets

,0,1
0,بعد استقالة رئيس #المحكمة_الدستورية ننتظر استق...,OBJ
1,أهنئ الدكتور أحمد جمال الدين، القيادي بحزب مصر...,POS
2,البرادعي يستقوى بامريكا مرةاخرى و يرسل عصام ال...,NEG
3,#الحرية_والعدالة | شاهد الآن: #ليلة_الاتحادية ...,OBJ
4,الوالدة لو اقولها بخاطري حشيشة تضحك بس من اقول...,NEUTRAL
...,...,...
9689,والغاز مش مدعوم يا إنسان؟ وماذا عن الأسمنت وال...,NEG
9690,اغلاق كل الساحات والميادين الكبرى لمنع صلاة ال...,NEG
9691,"#الشروق ""الداخلية"": 400 ألف مواطن تقدموا لأداء...",OBJ
9692,#هتحبك_لو صحتها من النوم علشان تقولها بحبك ;),POS


In [81]:
#dropping OBJ label
df_tweets.drop(df_tweets[df_tweets[1] == 'OBJ'].index, inplace=True)

In [82]:
df_tweets[1].value_counts()

NEG        1642
NEUTRAL     805
POS         777
Name: 1, dtype: int64

In [83]:
#changing labels name
cols = ["tweet", "class"]
df_tweets.columns = cols
df_ArSAS.columns = cols

In [84]:
df_ArSAS

,tweet,class
0,المباراة القـادمة #غانا x #مصر الجولة الأخيرة ...,Positive
1,هل هذه هي سياسة خارجيه لدوله تحترم نفسها والآخ...,Negative
2,وزير خارجية فرنسا عن منتدى شباب العالم: شعرت ب...,Positive
3,ومع السيسي و بشار و ايران و بن زايد و والا خلي...,Negative
4,أهداف مباراة غانا 0 مصر 1 تصفيات كأس العالم 20...,Neutral
...,...,...
19892,ملخص مباراة نيجيريا vs الجزائر تصفيات كأس 🏆 ال...,Neutral
19893,ايطاليا في السان سيرو لعبت 42 مباراة 🇮🇹 31 فوز...,Positive
19894,المُلحق المؤهل لكأس العالم 🇪🇺أوروبا🇪🇺 مباراة ا...,Neutral
19895,رسائل وتوصيات منتدى شباب العالم .. د. عبدالله ...,Neutral


In [85]:
#unifying the labels
values_ArSAS = {'Negative' : 'neg', 'Positive' : 'pos', 'Neutral' : 'neu'}
values_tweets = {'NEG' : 'neg', 'POS' : 'pos', 'NEUTRAL' : 'neu'}

In [86]:
df_tweets["class"] = df_tweets["class"].map(values_tweets)
df_ArSAS["class"] = df_ArSAS["class"].map(values_ArSAS)

In [87]:
df_tweets

,tweet,class
1,أهنئ الدكتور أحمد جمال الدين، القيادي بحزب مصر...,pos
2,البرادعي يستقوى بامريكا مرةاخرى و يرسل عصام ال...,neg
4,الوالدة لو اقولها بخاطري حشيشة تضحك بس من اقول...,neu
5,#انتخبوا_العرص #انتخبوا_البرص #مرسى_رئيسى #اين...,neu
6,امير عيد هو اللي فعلا يتقال عليه ستريكر صريح #...,pos
...,...,...
9688,مرسى لم يعد رئيسًا لخروجه على الشرعية وقراراته...,neg
9689,والغاز مش مدعوم يا إنسان؟ وماذا عن الأسمنت وال...,neg
9690,اغلاق كل الساحات والميادين الكبرى لمنع صلاة ال...,neg
9692,#هتحبك_لو صحتها من النوم علشان تقولها بحبك ;),pos


In [88]:
df_ArSAS

,tweet,class
0,المباراة القـادمة #غانا x #مصر الجولة الأخيرة ...,pos
1,هل هذه هي سياسة خارجيه لدوله تحترم نفسها والآخ...,neg
2,وزير خارجية فرنسا عن منتدى شباب العالم: شعرت ب...,pos
3,ومع السيسي و بشار و ايران و بن زايد و والا خلي...,neg
4,أهداف مباراة غانا 0 مصر 1 تصفيات كأس العالم 20...,neu
...,...,...
19892,ملخص مباراة نيجيريا vs الجزائر تصفيات كأس 🏆 ال...,neu
19893,ايطاليا في السان سيرو لعبت 42 مباراة 🇮🇹 31 فوز...,pos
19894,المُلحق المؤهل لكأس العالم 🇪🇺أوروبا🇪🇺 مباراة ا...,neu
19895,رسائل وتوصيات منتدى شباب العالم .. د. عبدالله ...,neu


In [89]:
df_cy

,tweet,class
0,' فينو الاهبل ابن الاهبل ',neg
1,' على المصرييييين وجمالهم ربنا يحميهم #MinaAtt...,pos
2,' @Kholoudkewan دول كتير اوى ودمهم خفيف العما...,pos
3,' انا بعد كده خلى اللى يوعنى بحاجه همضى على...,neg
4,' انا هنتحر ',neg
...,...,...
2741,' @wasfa_N الجمال مبيحتاح اي مكياج لناعم وله خ...,neu
2742,' @TheMurexDor نتمني وجود الفنانة رنا سماحة اف...,neu
2743,' ولد الهدى فالكائنات ضياء .. وفم الزمان تبسم ...,pos
2744,' @mohamed71944156 @samarroshdy1 انت متناقض جد...,neg


In [90]:
df_cy, test = train_test_split(df_cy, test_size=0.3)

In [91]:
train = pd.concat([df_cy,df_ArSAS, df_tweets], ignore_index=True)

In [92]:
train['class'].value_counts()

neg    9705
neu    8215
pos    5904
Name: class, dtype: int64

In [93]:
train['tweet'] = train['tweet'].apply(lambda x:   data_cleaning(x))

train.columns = ['tweet','class']

train['tweet'].head(20)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:19: DeprecationWarning: 'emoji.get_emoji_regexp()' is deprecated and will be removed in version 2.0.0. If you want to remove emoji from a string, consider the method emoji.replace_emoji(str, replace='').
To hide this warning, pin/downgrade the package to 'emoji~=1.6.3'


0     كاظم الساهر ﻓﺎﻛﻬﺔ ﺍﻟﺤﺐ ﺳﻴﺪﺗﻲﻛﻨﺖ ﺃﻫﻢ ﺍﻣﺮﺃﺓ ﻓﻲ ﺗ...
1     همسكهم الاتنين اخبطهم حيطة نبدا نضيف همس اتن ا...
2     الهم اشرح صدورنا بالقرءان الهم نور قبورنا بالق...
3     الصراحه انا مش عارفه اتعاطف معاها هى دلعها انا...
4                     اريد عمل اناسايق اريد عمل اناسايق
5     اومال ضاكتورة ايه بقى حسرة مسم اومال ضاكتور اي...
6     فحق الشرط لتستحق الوعد تنال المكافاة فحق شرط ل...
7     اليلو دراجونز حضروا ماتش الاسماعيلة النهارده ؟...
8                              اهم شي الطاعة اهم شء طاع
9     متشعلق بضوافري اترك ايتها المعشوقة مزيكتي متشع...
10    ربنا ينتقم ظالم معتقلي الاتحادية ربنا ينتقم ظا...
11    انك تبلغ الدين شييا توقر الخلايق تحتقر احدا فا...
12    عاملين ايه ياحلوين واحشني الصبح عندي امتحان بق...
13                  الناس ام دم خفيف ♥ ناس ام دم خفيف ♥
14    اة دة هدفك يعني القصة انك تعملي فتنة صح متخافي...
15    شكرا اله لانك تقبلني بكل سويي وزلاتي تقصيري لا...
16    ليها اسبابها ليا اسبابى لقدر اسباب نفسى اكمل ا...
17                اهدنا الصراط المستقيم اهد صراط

In [94]:
train['tweet']

0        كاظم الساهر ﻓﺎﻛﻬﺔ ﺍﻟﺤﺐ ﺳﻴﺪﺗﻲﻛﻨﺖ ﺃﻫﻢ ﺍﻣﺮﺃﺓ ﻓﻲ ﺗ...
1        همسكهم الاتنين اخبطهم حيطة نبدا نضيف همس اتن ا...
2        الهم اشرح صدورنا بالقرءان الهم نور قبورنا بالق...
3        الصراحه انا مش عارفه اتعاطف معاها هى دلعها انا...
4                        اريد عمل اناسايق اريد عمل اناسايق
                               ...                        
23819    مرسى يعد رييسا لخروجه الشرعية وقراراته ثورية م...
23820    والغاز مش مدعوم انسان؟ وماذا الاسمنت والحديد و...
23821    اغلاق الساحات والميادين الكبرى لمنع صلاة العيد...
23822    هتحبك صحتها النوم علشان تقولها بحبك هتحب صحت ن...
23823    شى كتبته غلط شى حسبته غلط فى الامتحانات شي كتب...
Name: tweet, Length: 23824, dtype: object

In [95]:
test['tweet'] = test['tweet'].apply(lambda x:   data_cleaning(x))

test.columns = ['tweet','class']

test['tweet'].head(20)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:19: DeprecationWarning: 'emoji.get_emoji_regexp()' is deprecated and will be removed in version 2.0.0. If you want to remove emoji from a string, consider the method emoji.replace_emoji(str, replace='').
To hide this warning, pin/downgrade the package to 'emoji~=1.6.3'


394     خاليه انا الحب فلاتخدعكم كتاباتى احبت شخص وفقد...
1537                        انتي اسمك ايه ؟ انت اسم ايه ؟
1451    بارك اله فيكم اخي شمالي … ربنا يستجيب دعاءكم ي...
1165    صباحك ويومك جميل انشاءلهمرسي لذوقك يوم جميل ان...
548     ايه خايف يقول وانا مش هقول الى حاسه انا بيه اي...
646     مش الاولتراس شباب جامعي انا معايا عربجية حرفيا...
2485    اخر الزمن مراقب عرص بيقولى مجبتش معاك مسطرة لي...
2321    اله يمسي خاطرك بالسعاده يامسعد قلبي بحلو السوا...
2469    القرانيه واضحه وخاصه قوله جمعهم اذا شاء قدير ا...
1661    ذكريات حياتك مينفعش تقال لحد مش عشان غلط ، عيب...
1343                         ان معي ربي سيهدين ان رب سيهد
687     علمتني الحياه ان يعيشون الارض ملايكة بشر يصيبو...
481     الحمد مستنية الامتحانات تيجى عشان الاجازة بقى ...
1835                         ماشي امر افندم ماش امر افندم
2532    اوقات بيعدي يومك لحظات تبه نفسك تعملها وتعدها ...
365     افضل الصور اﻻرسناليه فى هشتاق جديد الكل ينزل ا...
220     واحلي تحيه لاولاد بلدي اجمد ناس المصرين وصحابه...
1906    فى مشك

In [96]:
run_models(LinearSVC(), alg_name, analyzer= "char", ngram=6)
        

classifier: LinearSVC
------------------------------------
The results accuracy on the training data 0.9868200134318335
The Cross validation Accuracy is 0.7413098589117642, The F1-score is 0.7313054773513221
The Results on test data
accuracy: 
0.6310679611650486
f1_score: 
0.613162600978536


It seems We reached an end to machine learning algorithm we will try get better results using Deep Learning

## Best results 
#accuracy 0.63
#f1-score 0.62

#AS the difference between the dev and the Test is very large that suggestes that, they have a different distribution so we will not use extral data in the DL approach.
